In [37]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, LinearColorMapper, LogColorMapper
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Colorblind, Magma256

output_notebook()
palette = Colorblind[8]
palette_gradient = Magma256

Loading BokehJS ...

In [30]:
df_iterations = pd.read_csv("results.20220312-204209.duet.csv")

In [31]:
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])["iteration_time_ns"].transform(pd.Series.cumsum)
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])["iteration_start_ns"].shift(1, fill_value=0)
df_iterations["iteration_start_ns"] = df_iterations["iteration_start_ns"] + df_iterations["epoch_start_ms"] * 1000

df_iterations["iteration_end_ns"] = df_iterations["iteration_start_ns"] + df_iterations["iteration_time_ns"]

df_iterations

,benchmark,wallclock_start_ms,epoch_start_ms,iteration_time_ns,iteration,machine,provider,jdk,jdk_version,time,pair,kind,total_ms,process_cpu_time_ns,compilation_time_ms,compilation_total_ms,iteration_start_ns,iteration_end_ns
0,chi-square,1647113929416,1647113929416,7922862709,0,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.416Z,A,NaN,NaN,NaN,NaN,68846,1647113929416000,1647121852278709
1,chi-square,1647113929416,1647113929416,2797235407,1,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.416Z,A,NaN,NaN,NaN,NaN,68846,1647121852278709,1647124649514116
2,chi-square,1647113929416,1647113929416,2298627066,2,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.416Z,A,NaN,NaN,NaN,NaN,68846,1647124649514116,1647126948141182
3,chi-square,1647113929416,1647113929416,1734212925,3,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.416Z,A,NaN,NaN,NaN,NaN,68846,1647126948141182,1647128682354107
4,chi-square,1647113929416,1647113929416,2005866261,4,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.416Z,A,NaN,NaN,NaN,NaN,68846,1647128682354107,1647130688220368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,chi-square,1647113929413,1647113929413,1616461266,95,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.413Z,B,NaN,NaN,NaN,NaN,67768,1647278202341977,1647279818803243
196,chi-square,1647113929413,1647113929413,1341146331,96,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.413Z,B,NaN,NaN,NaN,NaN,67768,1647279818803243,1647281159949574
197,chi-square,1647113929413,1647113929413,1238909940,97,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.413Z,B,NaN,NaN,NaN,NaN,67768,1647281159949574,1647282398859514
198,chi-square,1647113929413,1647113929413,1256423493,98,NaN,NaN,OpenJDK Runtime Environment,11.0.11+9,2022-03-12T19:38:49.413Z,B,NaN,NaN,NaN,NaN,67768,1647282398859514,1647283655283007


In [32]:
source = ColumnDataSource(df_iterations)
pair_cmap = factor_cmap("pair", palette=palette, factors=sorted(df_iterations["pair"].unique()))
p = figure()
p.circle(x="iteration_start_ns", y="iteration", color=pair_cmap, source=source)
p.cross(x="iteration_end_ns", y="iteration", color=pair_cmap, source=source)

show(p)

In [33]:
columns_of_interest = ["benchmark", "pair", "iteration", "iteration_time_ns", "iteration_start_ns", "iteration_end_ns"]
df = df_iterations[columns_of_interest]

# Compute overlap interval
df_overlap = df[df.pair == "A"].merge(df[df.pair == "B"], on=["benchmark"], suffixes=["_A", "_B"], how="inner")
df_overlap["overlap_start_ns"] = df_overlap[["iteration_start_ns_A", "iteration_start_ns_B"]].max(axis=1)
df_overlap["overlap_end_ns"] = df_overlap[["iteration_end_ns_A", "iteration_end_ns_B"]].min(axis=1)

# Filter out non overlapping intervals
df_overlap["overlap"] = df_overlap["overlap_start_ns"].lt(df_overlap["overlap_end_ns"])
df_overlap = df_overlap[df_overlap["overlap"] == True]
df_overlap.reset_index(drop=True, inplace=True)

# Compute overlap properties
df_overlap["overlap_size"] = df_overlap["overlap_end_ns"] - df_overlap["overlap_start_ns"]


In [38]:
source = ColumnDataSource(df_overlap)
p = figure()
p.hbar(y="index", left="overlap_start_ns", right="overlap_end_ns", source=source,
       color={"field": "overlap_size", "transform": LogColorMapper(palette=palette_gradient)})

show(p)

In [35]:
source = ColumnDataSource(df_overlap)
p = figure()
p.vbar(x="overlap_start_ns", top="overlap_size", source=source)

show(p)